In [ ]:
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, make_scorer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier
import matplotlib.pyplot as plt
from functools import partial


In [ ]:
categories = ['NORMAL', 'PNEUMONIA']
target_size = (150, 150)
images = []
labels = []

def preprocess_image(img):
    img = img.convert('L')  # Convert to grayscale
    img = img.resize(target_size)  # Resize image
    img_array = np.array(img)  # Convert to numpy array
    return img_array

def load_data(data_dir):
    for label, category in enumerate(categories):
        category_dir = os.path.join(data_dir, category)
        for filename in os.listdir(category_dir):
            if filename.endswith('.jpeg'):
                img_path = os.path.join(category_dir, filename)
                with Image.open(img_path) as img:
                    img_array = preprocess_image(img)
                    images.append(img_array)
                    labels.append(label)

def create_model(optimizer='adam', dropout_rate=0.0):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    # model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['recall'])
    return model



def main():
    print("Loading data...")
    load_data('./chest_Xray/DATA')
    print("Data loaded")

    X = np.array(images)
    y = np.array(labels)

    # Normalize pixel values to between 0 and 1
    X = X / 255.0

    # Reshape for CNN
    X = X.reshape(-1, target_size[0], target_size[1], 1)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Emballer le modèle Keras avec KerasClassifier
    # create_model_partial = partial(create_model, optimizer='adam')

    model = KerasClassifier(build_fn=create_model, verbose=0)
    # Définir la grille des hyperparamètres

    param_dist = {
        'optimizer': ['adam', 'rmsprop'],
        # 'dropout_rate': [0.0, 0.2, 0.5],
        'batch_size': [16, 32, 64],
        'epochs': [10, 20]
    }


    def recall_scorer(y_true, y_pred):
        return recall_score(y_true, y_pred)

    # Créer un scorer sklearn pour la recherche aléatoire
    recall = make_scorer(recall_scorer, greater_is_better=True)

    # Initialiser RandomizedSearchCV avec scorer personnalisé
    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3, scoring=recall, verbose=2, n_jobs=-1)

    # Exécuter la recherche aléatoire
    random_result = random_search.fit(X_train, y_train)

    # Afficher les meilleurs hyperparamètres
    print(f"Best: {random_result.best_score_} using {random_result.best_params_}")

    # Utiliser les meilleurs hyperparamètres pour évaluer le modèle sur l'ensemble de validation
    best_model = random_result.best_estimator_
    val_score = best_model.score(X_test, y_test)
    print(f"Validation recall: {val_score}")


main()